In [43]:
import pandas as pd
import numpy as np
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Dropout
from tensorflow.keras.layers import Dense

from sklearn.metrics import confusion_matrix, accuracy_score,classification_report

In [8]:
df = pd.read_csv('train.csv')
df.dropna()
df.reset_index()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...,...
20795,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [9]:
X = df.drop('label',axis=1)

In [10]:
y = df['label']

In [11]:
print(X.shape,y.shape)

(20800, 4) (20800,)


# OneHot Rep

In [12]:
vocab_size = 10000
messages = X.copy()

### Data Preprocessing

In [15]:
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-z]', ' ', str(messages['title'][i]))
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [16]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic [photos]',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find 

In [17]:
onehot_repr = [one_hot(words,vocab_size,)for words in corpus]
onehot_repr

[[4693, 1736, 769, 9078, 7189, 6832, 7654, 9871, 3767, 6053],
 [6086, 9125, 3280, 4144, 8914, 8774, 6219],
 [4181, 20, 450, 1931],
 [5104, 1944, 3230, 5917, 9393, 9784],
 [4504, 8914, 1760, 187, 9252, 6776, 8914, 3278, 2257, 850],
 [2745,
  1465,
  4176,
  3813,
  2903,
  7504,
  1937,
  4152,
  489,
  7894,
  3466,
  5082,
  3171,
  7312,
  6219],
 [8691, 8691, 6099, 5648, 2679, 2705, 9425, 9934, 6296, 7207, 5939, 551],
 [8850, 9840, 8978, 3789, 3007, 2271, 6728, 7215, 2130, 7381, 4136],
 [5186,
  7465,
  5403,
  1696,
  7504,
  2799,
  3811,
  1334,
  6227,
  9809,
  2130,
  7381,
  4136],
 [2417, 2990, 5193, 4051, 2746, 8583, 7504, 6093, 2130, 7381, 4136],
 [9851, 1039, 9581, 6042, 4812, 7786, 6626, 8512, 7504, 7318],
 [4195, 1279, 6921, 2539, 3623, 2305, 6074, 817],
 [4645, 4645, 9219, 7362, 5337, 7637, 1875, 5586, 2698, 2032, 2600],
 [5917, 5731, 7189, 7786, 7504, 2746],
 [115, 5562, 9853, 8553, 9127, 6227, 1188, 3484, 5870],
 [5758, 8340, 8584, 373, 233, 3834, 8438, 2130, 7381, 4

# Embedding Representation

In [19]:
sent_length = 20 #Max sent lenght
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen= sent_length) #pre -> add zeroes before 
print(embedded_docs)

[[   0    0    0 ... 9871 3767 6053]
 [   0    0    0 ... 8914 8774 6219]
 [   0    0    0 ...   20  450 1931]
 ...
 [   0    0    0 ... 2130 7381 4136]
 [   0    0    0 ... 9736 6161 5193]
 [   0    0    0 ... 4357 6334 6124]]


In [27]:
X_final = np.array(embedded_docs)
Y_final = np.array(y)

In [45]:
embedding_vector_features=40
model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 20, 40)            400000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [46]:
X_train,X_test,Y_train,Y_test = train_test_split(X_final,Y_final,test_size=0.33,random_state=42)

In [47]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Epoch 1/10
218/218 [==============================] - 4s 19ms/step - loss: 0.3153 - accuracy: 0.8639 - val_loss: 0.1798 - val_accuracy: 0.9285
Epoch 2/10
218/218 [==============================] - 4s 18ms/step - loss: 0.1149 - accuracy: 0.9565 - val_loss: 0.2009 - val_accuracy: 0.9304
Epoch 3/10
218/218 [==============================] - 4s 18ms/step - loss: 0.0763 - accuracy: 0.9731 - val_loss: 0.2090 - val_accuracy: 0.9293
Epoch 4/10
218/218 [==============================] - 4s 18ms/step - loss: 0.0498 - accuracy: 0.9844 - val_loss: 0.2205 - val_accuracy: 0.9267
Epoch 5/10
218/218 [==============================] - 4s 18ms/step - loss: 0.0333 - accuracy: 0.9891 - val_loss: 0.2875 - val_accuracy: 0.9248
Epoch 6/10
218/218 [==============================] - 4s 18ms/step - loss: 0.0214 - accuracy: 0.9943 - val_loss: 0.3245 - val_accuracy: 0.9288
Epoch 7/10
218/218 [==============================] - 4s 18ms/step - loss: 0.0130 - accuracy: 0.9957 - val_loss: 0.3905 - val_accuracy: 0.9256

# Perfomance

In [48]:
y_pred = model.predict_classes(X_test)
print(y_pred)

[[1]
 [1]
 [0]
 ...
 [0]
 [0]
 [0]]


In [49]:
cm = confusion_matrix(Y_test,y_pred)
acc = accuracy_score(Y_test,y_pred)

In [50]:
cm,acc

(array([[3121,  328],
        [ 200, 3215]], dtype=int64),
 0.9230769230769231)